In [9]:
import openai_api as opai
from create_hed_prompts import get_hed_vocab

## From HED tags to description

In [17]:
messages=[
        {"role": "system", "content": "You are a good reporter."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'The foreground view consists of a high number of ingestible objects. \nThe background view features an adult human body, outdoors, with various furnishings, natural features such as the sky, urban elements, and man-made objects.'

In [18]:
messages=[
        {"role": "system", "content": "You are a good writer."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'In the foreground view, there is a high count of ingestible objects, indicating a variety of food or drinks. In the background view, there is an adult human body, outdoors, with various furnishing and natural features such as the sky. Additionally, there are man-made objects characteristic of an urban environment.'

In [19]:
messages=[
        {"role": "system", "content": "You are a concise translator."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'In the foreground, there is a high count of ingestible objects. In the background, there is an adult human body, outdoors, with a combination of natural features such as the sky and urban elements including man-made objects and furnishings.'

## From description to HED tags

Retrieve HED terms from HEDLatest.xml on Github

In [12]:
hed_vocab = get_hed_vocab()
print(hed_vocab)

Event,Sensory-event,Agent-action,Data-feature,Experiment-control,Experiment-procedure,Experiment-structure,Measurement-event,Agent,Animal-agent,Avatar-agent,Controller-agent,Human-agent,Robotic-agent,Software-agent,Action,Communicate,Communicate-gesturally,Clap-hands,Clear-throat,Frown,Grimace,Nod-head,Pump-fist,Raise-eyebrows,Shake-fist,Shake-head,Shhh,Shrug,Smile,Spread-hands,Thumbs-down,Thumb-up,Wave,Widen-eyes,Wink,Communicate-musically,Hum,Play-instrument,Sing,Vocalize,Whistle,Communicate-vocally,Cry,Groan,Laugh,Scream,Shout,Sigh,Speak,Whisper,Move,Breathe,Blow,Cough,Exhale,Hiccup,Hold-breath,Inhale,Sneeze,Sniff,Move-body,Bend,Dance,Fall-down,Flex,Jerk,Lie-down,Recover-balance,Sit-down,Sit-up,Stand-up,Stretch,Shudder,Stumble,Turn,Move-body-part,Move-eyes,Blink,Close-eyes,Fixate,Inhibit-blinks,Open-eyes,Saccade,Squint,Stare,Move-face,Bite,Burp,Chew,Gurgle,Swallow,Gulp,Yawn,Move-head,Lift-head,Lower-head,Turn-head,Move-lower-extremity,Curl-toes,Hop,Jog,Jump,Kick,Pedal,Press-foot,Run

In [11]:
messages=[
        {"role": "system", "content": "You are a good translator."},
        {"role": "system", "content": f"Use only terms from this comma-separated list: {hed_vocab}"},
        {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
        {"role": "user", "content": "A visual presentation of an image of an open market full of people and piles of vegetables."}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'(Visual-presentation,(2D-view,(Background-view,Open-market),(Foreground-view,((Center-of,Image),(People,Pile-of-vegetables))))'

In [14]:
messages=[
        {"role": "system", "content": f"You are a precise translator using only terms from this comma-separated list: {hed_vocab}"},
        {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
        {"role": "user", "content": "A visual presentation of an image of an open market full of people and piles of vegetables."}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'(Visual-presentation, (Background-view, Outdoors), (Foreground-view, (Image, Open-market, Full-of-people, Piles-of-vegetables)))'